<a href="https://colab.research.google.com/github/EduardMyalkin/PracticaSHTC-411/blob/main/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2_%D0%9A%D0%BB%D0%B5%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D0%B9_%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82_%D0%9D%D0%B0%D0%B3%D0%B5%D0%BB%D1%8F_%D0%A8%D1%80%D0%B5%D0%BA%D0%B5%D0%BD%D0%B1%D0%B5%D1%80%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
from typing import List, Tuple

def nagel_schreckenberg_model(Nc: int, positions: List[int], vmax: int, p: float, steps: int = 30):
    """
    Модель Нагеля-Шрекенберга для дорожного движения

    Parameters:
    Nc - количество машин
    positions - стартовые позиции машин (должны быть отсортированы по возрастанию)
    vmax - максимальная скорость
    p - вероятность замедления
    steps - количество итераций
    """

    # Проверка корректности входных данных
    if len(positions) != Nc:
        raise ValueError("Количество позиций должно совпадать с количеством машин")

    # Инициализация скоростей (все начинают с скорости 0)
    velocities = [0] * Nc

    # Длина дороги (предполагаем кольцевую дорогу длиной 30)
    road_length = 30

    # Результаты для каждой итерации
    results = []

    print(f"{'Шаг t':<6} {'Позиции':<30} {'Скорости':<30} {'Пересечения 0':<15}")
    print("-" * 85)

    for t in range(steps + 1):
        # Собираем информацию о пересечениях нулевой клетки на этом шаге
        zero_crossings = []

        # Вывод текущего состояния
        pos_str = "[" + ", ".join(f"{pos:2d}" for pos in positions) + "]"
        vel_str = "[" + ", ".join(f"{vel:2d}" for vel in velocities) + "]"

        # Для шага t > 0 выполняем итерацию модели
        if t > 0:
            new_positions = [0] * Nc
            new_velocities = [0] * Nc

            for i in range(Nc):
                # 1. Ускорение: v = min(v + 1, vmax)
                velocities[i] = min(velocities[i] + 1, vmax)

                # 2. Замедление: учитываем расстояние до впереди идущей машины
                next_car_index = (i + 1) % Nc
                distance = (positions[next_car_index] - positions[i] - 1) % road_length

                # Если следующая машина впереди, учитываем расстояние до неё
                if positions[next_car_index] > positions[i]:
                    distance = positions[next_car_index] - positions[i] - 1
                else:
                    # Если следующая машина "переехала" через 0
                    distance = (positions[next_car_index] + road_length) - positions[i] - 1

                velocities[i] = min(velocities[i], distance)

                # 3. Случайное замедление: с вероятностью p уменьшаем скорость на 1
                if random.random() < p and velocities[i] > 0:
                    velocities[i] = max(velocities[i] - 1, 0)

                # 4. Движение: обновляем позицию
                new_pos = (positions[i] + velocities[i]) % road_length
                new_positions[i] = new_pos

                # Проверяем пересечение нулевой клетки
                old_pos = positions[i]
                if old_pos + velocities[i] >= road_length:
                    zero_crossings.append(i)

            positions = new_positions
        else:
            # Для шага t=0 просто выводим начальное состояние
            zero_crossings = []

        # Форматируем пересечения нулевой клетки
        crossings_str = "[" + ", ".join(map(str, zero_crossings)) + "]" if zero_crossings else "[]"

        print(f"{t:<6} {pos_str:<30} {vel_str:<30} {crossings_str:<15}")

        results.append({
            'step': t,
            'positions': positions.copy(),
            'velocities': velocities.copy(),
            'zero_crossings': zero_crossings.copy()
        })

    return results

# Пример использования
if __name__ == "__main__":
    # Параметры модели
    Nc = 10  # количество машин
    start_positions = [0, 3, 5, 8, 11, 14, 17, 20, 23, 26]  # стартовые позиции (должны быть отсортированы!)
    vmax = 3  # максимальная скорость
    p = 0.05   # вероятность замедления

    # Запуск модели
    results = nagel_schreckenberg_model(Nc, start_positions, vmax, p)

    # Дополнительная статистика
    print("\n" + "="*50)
    print("СТАТИСТИКА ПОСЛЕ 30 ШАГОВ:")
    print("="*50)

    avg_speed = sum(results[-1]['velocities']) / Nc
    max_speed = max(results[-1]['velocities'])
    min_speed = min(results[-1]['velocities'])

    print(f"Средняя скорость: {avg_speed:.2f}")
    print(f"Максимальная скорость: {max_speed}")
    print(f"Минимальная скорость: {min_speed}")

    # Подсчет общего количества пересечений нулевой клетки
    total_crossings = sum(len(step['zero_crossings']) for step in results)
    print(f"Всего пересечений нулевой клетки: {total_crossings}")
    import pandas as pd

def create_results_table(results):
    """Создает таблицу Pandas DataFrame с результатами"""
    data = []
    for step_data in results:
        step = step_data['step']
        positions = step_data['positions']
        velocities = step_data['velocities']
        crossings = step_data['zero_crossings']

        data.append({
            'Шаг t': step,
            'Позиции': positions.copy(),
            'Скорости': velocities.copy(),
            'Пересечения 0': crossings.copy()
        })

    return pd.DataFrame(data)

# Использование с таблицей
df = create_results_table(results)
print("\nТаблица результатов:")
print(df.to_string(index=False))


Шаг t  Позиции                        Скорости                       Пересечения 0  
-------------------------------------------------------------------------------------
0      [ 0,  3,  5,  8, 11, 14, 17, 20, 23, 26] [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0] []             
1      [ 0,  3,  5,  8, 11, 14, 17, 20, 23, 26] [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0] []             
2      [ 1,  4,  6,  9, 12, 15, 18, 21, 24, 27] [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1] []             
3      [ 2,  5,  7, 11, 14, 17, 20, 23, 26, 29] [ 1,  1,  1,  2,  2,  2,  2,  2,  2,  2] [9]            
4      [ 4,  6,  9, 13, 16, 19, 22, 25, 28,  1] [ 2,  1,  2,  2,  2,  2,  2,  2,  2,  2] [8]            
5      [ 5,  8, 12, 15, 18, 20, 24, 27,  0,  3] [ 1,  2,  3,  2,  2,  1,  2,  2,  2,  2] []             
6      [ 6, 11, 14, 17, 19, 22, 26, 29,  2,  4] [ 1,  3,  2,  2,  1,  2,  2,  2,  2,  1] [7]            
7      [ 8, 13, 16, 18, 21, 25, 28,  1,  3,  5] [ 2,  2,  2,  1,  2,  3,  2,  2,  1,  1] [6]  